In [ ]:
import pandas as pd
import plotly.express as px
import json
import plotly.graph_objects as go

In [ ]:
new = pd.read_csv('results.csv', index_col=0)
paper_results = pd.read_excel('data/mdevs_solutions.xlsx', sheet_name=["results_regular_BCH", "results_regular_CPLEX", "results_large_BCH"])
cplex = paper_results["results_regular_CPLEX"]
bch = paper_results["results_regular_BCH"]
large = paper_results["results_large_BCH"]
large["ID_instance"] = large["ID_instance"].apply(lambda x:  50+x)

bch = bch[bch["battery_charging"] == "constant-time"]
cplex = cplex[cplex["battery_charging"] == "constant-time"]
large = large[large["battery"] == "constant-time"]
# Add a method column for each dataframe
new["method"] = new.apply(lambda x: "fragment", axis=1)
cplex["method"] = cplex.apply(lambda x: "cplex", axis=1)
bch["method"] = cplex.apply(lambda x: "bch", axis=1)
large["method"] = large.apply(lambda x: "bch", axis=1)
new.rename(columns={"objective": "objective_value"}, inplace=True)


In [ ]:
def get_instance_id(label: str):
    try:
        data = json.load(open(f"data/instances_regular/{label}.json"))
        return data["ID"]
    except:
        print(f"No data for instance {label} found", label[-1])
        return 50 + int(label[-1])
    
new["ID_instance"] = new["label"].apply(get_instance_id)



In [ ]:
new["ID_instance"]

In [ ]:
new = new[["ID_instance", "method", "objective_value", "runtime"]]
cplex = cplex[["ID_instance", "method", "objective_value", "runtime"]]
bch = bch[["ID_instance", "method", "objective_value", "runtime"]]
large = large[["ID_instance", "method", "objective_value", "runtime"]]
#concatenate these dfs
result_df = pd.concat([new, cplex, bch, large])
result_df.groupby(["ID_instance"])

differences = []
{
    "ID": ...,
    "runtime_diff": ...,
    "comparator": "fragment-cplex"
}
for name, group in result_df.groupby(["ID_instance"]):
    name = name[0]
    # if name > 50:
    #      print(name)
    #      print(group)
    # else:
    #     continue
    # print(name)
    # assert bch and fragment have the same objective value
    try:
        fragment_runtime = group[group["method"] == "fragment"]["runtime"].values[0]
        bch_runtime = group[group["method"] == "bch"]["runtime"].values[0]
        assert group[group["method"] == "fragment"]["objective_value"].values[0] == group[group["method"] == "bch"]["objective_value"].values[0]

        differences += [
            {
                "ID": name,
                "runtime_diff": fragment_runtime - bch_runtime,
                "runtime_percentage": (fragment_runtime - bch_runtime )/ bch_runtime * 100,
                "comparator": "fragment-bch"
            }
        ]
        
        # differences += [
        #     {
        #         "ID": name,
        #         "runtime_diff": fragment_runtime - group[group["method"] == "cplex"]["runtime"].values[0],
        #         "runtime_percentage": (fragment_runtime - group[group["method"] == "cplex"]["runtime"].values[0]) / group[group["method"] == "cplex"]["runtime"].values[0]*100,
        #         "comparator": "fragment-cplex"
        #     },
        # ]
    except:
        print("missing data for instance", name)
   
    #compute the difference between the method column for fragment and cplex, fragment and bch
 

In [ ]:
result_df[51 == result_df["ID_instance"]]

In [ ]:
diff_df = pd.DataFrame(differences)
len(diff_df)
diff_df["ID"]
px.scatter(diff_df, x="ID", y="runtime_diff", color="comparator", title="Runtime change from paper methods").show()

In [ ]:
px.scatter(diff_df, x="ID", y="runtime_percentage", color="comparator", title="Runtime % change in fragment from paper methods").show()

In [ ]:
fragment_file = r'data/instances_regular/fragments/f-I-5-5-200-10.json'
fragment_file = r'data/instances_large/fragments/f-I-7-7-1000-01.json'
fragment_data = json.load(open(fragment_file))
# instance_data = json.load(open('data/instances_regular/I-1-1-50-01.json'))
print(len(fragment_data['fragments']), len(fragment_data['contracted_fragments']),  len(fragment_data['contracted_fragments'])/len(fragment_data['fragments']) *100)

In [ ]:
from utils.visualiser import visualise_timed_network
from constants import TimedDepot

### Arrival cases


In [ ]:
# Create 3 timed depots with same id, 5 time units apart
timed_depots = [TimedDepot(time=0, id=0), TimedDepot(id=0, time=5), TimedDepot(time=10, id=0), TimedDepot(time=15, id=0)]
fig = visualise_timed_network(timed_depots, [], [{"timed_depot": a} for a in zip(timed_depots[:-1], timed_depots[1:])])

#compressed
fig = visualise_timed_network(timed_depots[:1]+timed_depots[-1:], [], [{"timed_depot": a} for a in zip(timed_depots[:-1], timed_depots[1:])])
inbound = False
for td in timed_depots:
    # inbound
    # if inbound:
    if td.time > 9:
        x, y, ax, ay = td.time+3, td.id+2, td.time, td.id
    else:
        x, y, ax, ay = td.time, td.id, td.time-3, td.id+2
    

    fig.add_annotation(
        x=x,
        y=y,
        ax=ax,
        ay=ay,
        xref='x',
        yref='y',
        axref='x',
        ayref='y',
        showarrow=True,
        arrowhead=2,
        arrowsize=2,  # Increase the size of the arrowhead
        arrowwidth=2,  # Increase the width of the arrow
        arrowcolor='#636363'
    )
    if td.time == 10:
        x, y, ax, ay = td.time, td.id, td.time-3, td.id+2
        # x, y, ax, ay = td.time+3, td.id+2, td.time, td.id
        fig.add_annotation(
            x=x,
            y=y,
            ax=ax,
            ay=ay,
            xref='x',
            yref='y',
            axref='x',
            ayref='y',
            showarrow=True,
            arrowhead=2,
            arrowsize=2,  # Increase the size of the arrowhead
            arrowwidth=2,  # Increase the width of the arrow
            arrowcolor='#636363'
        )

fig.write_image("images/time_space_compressed_two_in_one_correct.png")
fig.show()

In [ ]:
from fragment_generation import ConstantFragmentGenerator
from constants import *

In [ ]:
gen = ConstantFragmentGenerator("data/instances_regular/I-5-5-200-07.json")
gen.generate_fragments(file="data/instances_regular/fragments/f-I-5-5-200-07.json")
print(gen.fragments_by_id[2234])
gen.generate_timed_network()
gen.build_model()
prior_solution, solution_routes = gen.read_solution(instance_type="regular")
print(gen.timed_depots_by_fragment_id[7869])
print(gen.timed_depots_by_fragment_id[2234])

frags = gen.timed_fragments_by_timed_depot[TimedDepot(time=272,id=0)]

sol_route = gen.get_validated_timed_solution(prior_solution)


In [ ]:
sol_frags = [f for r in sol_route for f in r if isinstance(f, Fragment) and f.id in [a.id for a in frags]]
sol_frags

In [ ]:
for r in sol_route:
    if any(f in r for f in sol_frags):
        print(r)

In [ ]:
old_charge = pd.read_csv('new_results.csv', index_col=0)
fixed = pd.read_csv('fixed_charge_cost.csv', index_col=0)
fixed["method"] = fixed.apply(lambda x: "fixed_charge", axis=1)

In [ ]:
df = pd.concat([old_charge, fixed])

In [ ]:
groupedby = df.groupby(["label"])
for name, group in groupedby:
    print(name)
    # print(group)\
    # assert objective is the same

    # show the duifference in num fragments
    print("num fragments", group["num_fragments"].values)

    print("")

In [ ]:
df = pd.read_xml(open('data/instances_large.xml', 'r'))

In [ ]:
df
# check each column e_j, s_j, c_j tripStartLocations has the same length
for _, row in df.iterrows():
    print(len(row["tripStartLocations"]), len(row["tripEndLocations"]), len(row["e_j"]), len(row["s_j"]), len(row["c_j"]))
    print(type(row["e_j"]))

In [ ]:
from formulations.naive_ip import NaiveIP

ip = NaiveIP("data/instances_regular/I-1-1-50-08.json", params=dict(UNDISCRETISED_MAX_CHARGE=70))

In [ ]:
ip.generate_cost_matrices()
ip.generate_valid_charge_levels()

In [ ]:
ids = [1, 6, 17, 26, 42, 1]
print(sum(ip.charge_matrix[s][e] for s,e in zip(ids, ids[1:])))# + 
sum(ip.location_by_id[j].charge for j in ids[1:-1])


In [ ]:
# Checking the max charge calcs are valid
from constants import *
locations = [ip.location_by_id[i] for i in ids]
# ip.generate_valid_charge_levels()
charge_dict = ip.max_charge_by_loc_pair_charge
charge = 140
prev = locations[0]
print(prev.offset_id, charge)
total_charge = ip.charge_matrix[prev.offset_id][locations[1].offset_id]
for next in locations[1:]:
    charge = charge_dict[prev, next, charge]
    total_charge += next.charge if isinstance(next, Job) else 0
    total_charge += ip.charge_matrix[prev.offset_id][next.offset_id]
    print(next.offset_id, charge)
    print(f"Metrics: {ip.charge_matrix[prev.offset_id][next.offset_id]}, {next.charge if isinstance(next, Job) else 0}")
    prev = next
print(total_charge)

In [ ]:
ip.location_by_id[25].charge

In [ ]:
for i,j in [(6,1), (1, 17)]:
    print(ip.time_matrix[i][j])

In [ ]:
df = pd.read_csv("data/results/sequencing_algorithm_results.csv")

In [ ]:
px.scatter(df, x="Jobs", y=["Recursion", "Forward Labelling"])

In [ ]:
df["Change"] = df.apply(lambda x: (-x["Recursion"]+x["Forward Labelling"])/x["Recursion"], axis=1)
px.scatter(df, x="Jobs", y="Change")